In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [11]:
wine_data = pd.read_csv('red.csv', delimiter=';')

In [ ]:
'''Train data generation'''
train, test = train_test_split(wine_data, test_size=0.2, shuffle=True, random_state=None)
